In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from minijpas_LF_and_puricomp import effective_volume
from my_functions import double_schechter

In [ ]:
%matplotlib inline

In [ ]:
# My LF

# nbs_list = [[1, 4], [4, 8], [8, 12], [12, 16]]
nbs_list = [[1, 4], [4, 8], [8, 12], [12, 16], [16, 20], [20, 24]]

survey_list = [f'minijpasAEGIS00{i}' for i in range(1, 4 + 1)] + ['jnep']
total_volume = 0
for [this_nb_min, this_nb_max] in nbs_list:
    total_volume += effective_volume(this_nb_min, this_nb_max, 'both')
L_binning = np.load('npy/L_nb_err_binning.npy')
b = np.log10(L_binning)
LF_bins = np.array([(b[i] + b[i + 1]) / 2 for i in range(len(b) - 1)])
bin_width = np.array([b[i + 1] - b[i] for i in range(len(b) - 1)])

qso_factor = 1

hist_mat = None
LF_raw = None
for i, [nb1, nb2] in enumerate(nbs_list):
    this_volume = effective_volume(nb1, nb2)
    this_hist = None
    for survey_name in survey_list:
        pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
        filename_hist = f'{pathname}/hist_i_mat_{survey_name}.npy'
        hist_i_mat = np.load(filename_hist)

        if this_hist is None:
            this_hist = hist_i_mat
        else:
            this_hist += hist_i_mat
    # When all 5 fields are added up, divide by total volume
    this_hist = this_hist / total_volume / bin_width

    if hist_mat is None:
        hist_mat = this_hist
    else:
        # hist_mat = np.vstack([hist_mat, this_hist])
        hist_mat = hist_mat + this_hist

    filename_dict = f'{pathname}/LFs.pkl'
    with open(filename_dict, 'rb') as file:
        if LF_raw is None:
            LF_raw = pickle.load(file)['LF_total_raw'] * this_volume
        else:
            LF_raw += pickle.load(file)['LF_total_raw'] * this_volume
    
L_LF_err_percentiles = np.percentile(hist_mat, [16, 50, 84], axis=0)
LF_err_plus = L_LF_err_percentiles[2] - L_LF_err_percentiles[1]
LF_err_minus = L_LF_err_percentiles[1] - L_LF_err_percentiles[0]
hist_median = L_LF_err_percentiles[1]

volwid = total_volume * bin_width
yerr_plus = (hist_median + volwid * (LF_err_plus) ** 2) ** 0.5 * volwid ** -0.5
yerr_minus = (hist_median + volwid * (LF_err_minus) ** 2) ** 0.5 * volwid ** -0.5

LF_dict = {
    'LF_bins': LF_bins,
    'LF_total': hist_median,
    'LF_total_uncorr': LF_raw / total_volume,
    'LF_total_err': [yerr_minus, yerr_plus]
}

In [ ]:
# Load reference LFs
pathname = '/home/alberto/almacen/literature_LF_data'

# Blanc 2011 (z=1.9--3.8)
filename = f'{pathname}/blanc2011_allz.txt'
df = pd.read_table(filename, delimiter='\t')
b11 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Blanc 2011 ($z=1.9-3.8$)',
    'fmt': 'h',
}

# Gronwall 2007 (z=3.1)
filename = f'{pathname}/gronwall2007_z3.1.txt'
df = pd.read_table(filename, delimiter='\t')
g07 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Gronwall 2007 ($z=3.1$)',
    'fmt': 'x'
}

# Konno 2016 (z=2.2)
filename = f'{pathname}/konno2016_z2.2_corrected.txt'
df = pd.read_table(filename, delimiter='\t')
k16 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Konno 2016 ($z=2.2$)',
    'fmt': '^'
}

# Matthee 2017 (z=2.2)
filename = f'{pathname}/matthee2017_z2.2.txt'
df = pd.read_table(filename, delimiter='\t')
m17a = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Matthee 2017 ($z=2.2$)',
    'fmt': '*'
}

# Matthee 2017 (z=2.4)
filename = f'{pathname}/matthee2017_z2.4.txt'
df = pd.read_table(filename, delimiter='\t')
m17b = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Matthee 2017 ($z=2.4$)',
    'fmt': '*'
}

# Ouchi 2008 (z=3.1)
filename = f'{pathname}/ouchi2008_z3.1.txt'
df = pd.read_table(filename, delimiter='\t')
u08 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Ouchi 2008 ($z=3.1$)',
    'fmt': 'o'
}

# Sobral 2016 (z=2.2)
filename = f'{pathname}/sobral2016_z2.2.txt'
df = pd.read_table(filename, delimiter='\t')
s16 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Sobral 2016 ($z=2.2$)',
    'fmt': 'D'
}

# Sobral 2017 (z=2.2)
filename = f'{pathname}/sobral2017_z2.2.txt'
df = pd.read_table(filename, delimiter='\t')
s17 = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Sobral 2017 ($z=2.2$)',
    'fmt': 'D'
}

# Sobral 2018 (z=2.5)
filename = f'{pathname}/sobral2018_z2.5.txt'
df = pd.read_table(filename, delimiter='\t')
s18a = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Sobral 2018 ($z=2.5$)',
    'fmt': 'D'
}

# Sobral 2018 (z=3.0)
filename = f'{pathname}/sobral2018_z3.0.txt'
df = pd.read_table(filename, delimiter='\t')
s18b = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Sobral 2018 ($z=3.0$)',
    'fmt': 'D'
}

# Sobral 2018 (z=3.2)
filename = f'{pathname}/sobral2018_z3.2.txt'
df = pd.read_table(filename, delimiter='\t')
s18c = {
    'logL': df['LogLya'],
    'Phi': df['phi'],
    'yerr_plus': df['phi_err_up'] - df['phi'],
    'yerr_minus': df['phi'] - df['phi_err_down'],
    'label': 'Sobral 2018 ($z=3.2$)',
    'fmt': 'D'
}

# Spinoso 2020
fnam = '/home/alberto/almacen/literature_LF_data/LF_data_SpinosoEtAl2020/20200518_J0395_SNR5_LFdata.txt'
loglya, ModLF, ModLF_pc16, ModLF_pc84 = np.genfromtxt(fnam, skip_header=3, usecols=(0,6,7,8), unpack=True)
dLogL = loglya[1] - loglya[0]
snr = ModLF / ModLF_pc16
ds20_z225 = {
    'logL': loglya[snr > 1],
    'Phi': ModLF[snr > 1],
    'yerr_plus': ModLF_pc84[snr > 1],
    'yerr_minus': ModLF_pc16[snr > 1],
    'label': 'Spinoso 2020 ($z=2.25$)',
    'fmt': 'X'
}

fnam = '/home/alberto/almacen/literature_LF_data/LF_data_SpinosoEtAl2020/20200518_J0410_SNR5_LFdata.txt'
loglya, ModLF, ModLF_pc16, ModLF_pc84 = np.genfromtxt(fnam, skip_header=3, usecols=(0,6,7,8), unpack=True)
dLogL = loglya[1] - loglya[0]
snr = ModLF / ModLF_pc16
ds20_z237 = {
    'logL': loglya[snr > 1],
    'Phi': ModLF[snr > 1],
    'yerr_plus': ModLF_pc84[snr > 1],
    'yerr_minus': ModLF_pc16[snr > 1],
    'label': 'Spinoso 2020 ($z=2.37$)',
    'fmt': 'X'
}

fnam = '/home/alberto/almacen/literature_LF_data/LF_data_SpinosoEtAl2020/20200518_J0430_SNR5_LFdata.txt'
loglya, ModLF, ModLF_pc16, ModLF_pc84 = np.genfromtxt(fnam, skip_header=3, usecols=(0,6,7,8), unpack=True)
dLogL = loglya[1] - loglya[0]
snr = ModLF / ModLF_pc16
ds20_z254 = {
    'logL': loglya[snr > 1],
    'Phi': ModLF[snr > 1],
    'yerr_plus': ModLF_pc84[snr > 1],
    'yerr_minus': ModLF_pc16[snr > 1],
    'label': 'Spinoso 2020 ($z=2.54$)',
    'fmt': 'X'
}

fnam = '/home/alberto/almacen/literature_LF_data/LF_data_SpinosoEtAl2020/20200518_J0515_SNR5_LFdata.txt'
loglya, ModLF, ModLF_pc16, ModLF_pc84 = np.genfromtxt(fnam, skip_header=3, usecols=(0,6,7,8), unpack=True)
dLogL = loglya[1] - loglya[0]
snr = ModLF / ModLF_pc16
ds20_z324 = {
    'logL': loglya[snr > 1],
    'Phi': ModLF[snr > 1],
    'yerr_plus': ModLF_pc84[snr > 1],
    'yerr_minus': ModLF_pc16[snr > 1],
    'label': 'Spinoso 2020 ($z=3.24$)',
    'fmt': 'X'
}

# Zhang 2021 (z=2.0-3.5)
df = pd.read_csv('csv/Zhang2021_LF.csv')
z21 = {
    'logL': df['Llya'],
    'Phi': df['Phi'],
    'yerr_plus': df['yerr_plus'] - df['Phi'],
    'yerr_minus': df['Phi'] - df['yerr_minus'],
    'label': 'Zhang 2021 ($z=2.0-3.5$)',
    'fmt': 'd'
}

# Liu 2022 (z=1.88-3.53)
df = pd.read_csv('csv/Liu_LF.csv')
l22 = {
    'logL': df['logLya'],
    'Phi': df['Phi'],
    'yerr_plus': df['yerr'],
    'yerr_minus': df['yerr'],
    'label': 'Liu 2022 ($z=1.9-3.5$)',
    'fmt': 'v'
}

# Assign colors
LF_ref_list = [b11, g07, k16, m17a, m17b, u08, s16, s17, s18a,
               s18b, s18c, ds20_z225, ds20_z237, ds20_z254,
               ds20_z324, z21, l22]
for i, lf_dict in enumerate(LF_ref_list):
    lf_dict['color'] = f'C{i}'

In [ ]:
from my_functions import schechter

fig, ax = plt.subplots(figsize=(8, 6))

# My LF
ax.errorbar(LF_dict['LF_bins'], LF_dict['LF_total'],
            yerr=LF_dict['LF_total_err'][:2],
            # xerr=bin_width*0.5,
            linestyle='', fmt='s',
            ecolor='k', markeredgecolor='k', markerfacecolor='red',
            markeredgewidth=1.5, markersize=8,
            capsize=4, label='This work (corrected)', zorder=99)

# My uncorr LF
ax.plot(LF_dict['LF_bins'], LF_dict['LF_total_uncorr'],
        marker='^', markerfacecolor='none', markeredgecolor='dimgray',
        markeredgewidth=2, markersize=8, ls='', label='This work (uncorrected)',
        zorder=97)

# Other LFs
# ref_LFs_to_plot = [k16, m17b, u08, s18a, s18b, s18c, ds20_z225, ds20_z237, ds20_z254,
#                    ds20_z324, z21, l22]
ref_LFs_to_plot = [l22, z21]
for i, lf in enumerate(ref_LFs_to_plot):
    ax.errorbar(lf['logL'], lf['Phi'],
                yerr=[lf['yerr_minus'], lf['yerr_plus']],
                linestyle='', fmt=lf['fmt'], c=lf['color'],
                alpha=0.8, label=lf['label'], capsize=3, zorder=i)

# Plot the reference LF curves
Lx = np.linspace(10 ** 42, 10 ** 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = double_schechter(
    Lx, phistar1, 10 ** Lstar1, alpha1, 10 ** phistar2, 10 ** Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='-.', alpha=0.7,
    label='Spinoso2020 ($z=2.2-3.25$)', zorder=51,
    color='C7'
)
# ax.plot(
#     np.log10(Lx), Phi_center * qso_factor, ls='-', alpha=1,
#     label='Input mock', zorder=51,
#     color='g'
# )

phistar1 = 10 ** -3.41
Lstar1 = 10 ** 42.87
alpha1 = -1.7

phistar2 = 10 ** -5.85
Lstar2 = 10 ** 44.6
alpha2 = -1.2

Phi_center = double_schechter(
    Lx, phistar1, Lstar1, alpha1, phistar2, Lstar2, alpha2
) * Lx * np.log(10)

ax.plot(
    np.log10(Lx), Phi_center, ls='--', alpha=0.7,
    label='Zhang2021 ($z=2-3.2$)', zorder=50,
    color='C8'
)

# My fit
# Lstar_fit = 44.61
# Phistar_fit = -6.03
# alpha = -1.5
# Lx = np.logspace(41, 47, 10000)
# Phi_center = schechter(Lx, 10**Phistar_fit, 10**Lstar_fit, alpha) * Lx * np.log(10)
# ax.plot(np.log10(Lx), Phi_center, zorder=98, c='r',
#         linewidth=2, label='This work best fit')

# Dummy to add text to legend
ax.plot([], [], ls='', label=r'$\bf{z = 2.0-4.1}$')

ax.set_ylim(1e-8, 1e-3)
ax.set_xlim(42.5, 45.5)
ax.set_yscale('log')

ax.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=15)
ax.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)', fontsize=15)

ax.tick_params(labelsize=14, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

handles, labels = ax.get_legend_handles_labels()
# order = np.array([3, 4, 0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1, 2])
order = np.array([3, 0, 4, 5, 6, 1, 2])
ax.legend(np.array(handles, dtype=object)[order], np.array(
    labels, dtype=object)[order], fontsize=8, ncol=1)

fig.savefig('figures/Combined_LF.pdf', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
from my_functions import central_wavelength, nb_fwhm, schechter


fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharey=True)

w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(np.arange(60))
w_lya = 1215.67

# My fit
Lstar_fit = 44.61
Phistar_fit = -6.03
alpha = -1.5
Phi_center = schechter(Lx, 10**Phistar_fit, 10**Lstar_fit, alpha) * Lx * np.log(10)

reference_LFs = [
    [s17, m17b, k16, ds20_z225, ds20_z237],
    [s18a, m17b, b11, ds20_z237],
    [s18b, u08, b11, ds20_z324],
    [s18c, s18b, u08, b11, ds20_z324],
    [b11, u08, s18b, s18c],
    [b11]
]

for i, ax in enumerate(axes.flatten()):
    [nb1, nb2] = nbs_list[i]
    
    z_min = (w_central[nb1] - nb_fwhm_Arr[nb1] * 0.5) / w_lya - 1
    z_max = (w_central[nb2] + nb_fwhm_Arr[nb2] * 0.5) / w_lya - 1

    this_hist = None
    this_volume = effective_volume(nb1, nb2, 'both')
    for survey_name in survey_list:
            pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
            filename_hist = f'{pathname}/hist_i_mat_{survey_name}.npy'
            hist_i_mat = np.load(filename_hist)

            if this_hist is None:
                this_hist = hist_i_mat
            else:
                this_hist += hist_i_mat
            filename_dict = f'{pathname}/LFs.pkl'
            with open(filename_dict, 'rb') as file:
                LF_raw = pickle.load(file)['LF_total_raw']

    L_LF_err_percentiles = np.percentile(this_hist, [16, 50, 84], axis=0)
    LF_err_plus = L_LF_err_percentiles[2] - L_LF_err_percentiles[1]
    LF_err_minus = L_LF_err_percentiles[1] - L_LF_err_percentiles[0]
    hist_median = L_LF_err_percentiles[1]

    yerr_plus = (hist_median + LF_err_plus **
                        2) ** 0.5 / bin_width / this_volume
    yerr_minus = (hist_median + LF_err_minus **
                        2) ** 0.5 / bin_width / this_volume


    this_LF_dict = {
        'LF_bins': LF_bins,
        'LF_total': hist_median / bin_width / this_volume,
        'LF_total_err': [yerr_minus, yerr_plus],
        'LF_total_uncorr': LF_raw,
    }

    ax.text(42.4, 1e-2, fr'$\bf z={z_min:0.1f}-{z_max:0.1f}$')
    
    ax.errorbar(this_LF_dict['LF_bins'], this_LF_dict['LF_total'],
            yerr=this_LF_dict['LF_total_err'][:2],
            # xerr=bin_width*0.5,
            linestyle='', fmt='s',
            ecolor='k', markeredgecolor='k', markerfacecolor='red',
            markeredgewidth=1, markersize=8,
            capsize=4, label='This work', zorder=99)
    ax.plot(this_LF_dict['LF_bins'], this_LF_dict['LF_total_uncorr'],
            marker='^', markerfacecolor='none', markeredgecolor='dimgray',
            markeredgewidth=1, markersize=8, ls='', label='This work (uncorrected)',
            zorder=97)

    for j, lf in enumerate(reference_LFs[i]):
        ax.errorbar(lf['logL'], lf['Phi'],
                    yerr=[lf['yerr_minus'], lf['yerr_plus']],
                    linestyle='', fmt=lf['fmt'], c=lf['color'],
                    alpha=0.8, label=lf['label'], capsize=3, zorder=i)

    if i > 2:
        ax.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=15)
    if i == 0 or i == 3:
        ax.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)', fontsize=15)

    ax.set_ylim(1e-8, 1e-1)
    ax.set_xlim(42.3, 45.7)
    ax.set_yscale('log')

    ax.tick_params(labelsize=14, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    ax.set_xticks(np.arange(42.5, 46, 0.5))

    ax.legend(fontsize=10, loc=0)

# fig.tight_layout()
fig.subplots_adjust(wspace=0.03, hspace=0.12)
fig.savefig('figures/Multi_LF.pdf', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
from plot_puricomp2d import load_puricomp1d

fig, axes = plt.subplots(2, 3, figsize=(15, 7), sharey=True)

comp_den_def = None
for i, ax in enumerate(axes.flatten()):
    [nb1, nb2] = nbs_list[i]

    z_min = (w_central[nb1] - nb_fwhm_Arr[nb1] * 0.5) / w_lya - 1
    z_max = (w_central[nb2] + nb_fwhm_Arr[nb2] * 0.5) / w_lya - 1

    # A text with the redshift interval
    ax.set_title(fr'$\bf z={z_min:0.1f}-{z_max:0.1f}$')

    this_dirname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'

    comp_list, _, _, puri_list, comp_den_list, _, _, puri_den_list, puricomp_bins = \
        load_puricomp1d(this_dirname)

    # Define the survey list in order
    survey_list = [f'AEGIS00{i}' for i in range(1, 4 + 1)] + ['J-NEP']

    # Bin centers
    bc = [puricomp_bins[i: i + 2].sum() * 0.5 for i in range(len(puricomp_bins) - 1)]

    # Plot the individual comps
    for j, comp in enumerate(comp_list):
        ax.plot(bc, comp, ls=':', alpha=0.6, marker='s', markersize=10,
                label=survey_list[j], c=f'C{j + 2}')

    # Total comp
    total_comp_num = (np.array(comp_list) *
                      np.array(comp_den_list)).sum(axis=0)
    total_comp_den = np.array(comp_den_list).sum(axis=0)
    total_comp_err = (total_comp_num / total_comp_den ** 2 +
                      total_comp_num ** 2 / total_comp_den ** 4 * total_comp_num) ** 0.5
    total_comp_err[~np.isfinite(total_comp_err)] = 0
    if comp_den_def is None:
        comp_den_def = total_comp_den
        comp_num_def = total_comp_num
    else:
        comp_den_def += total_comp_den
        comp_num_def += total_comp_num
    total_comp = total_comp_num / total_comp_den
    total_comp[~np.isfinite(total_comp)] = 0
    ax.errorbar(bc, total_comp, ls='-', fmt='s', c='k',
                markersize=9, yerr=total_comp_err,
                capsize=4)
    # Dummy point
    ax.errorbar([], [], markersize=9, c='k', label='Total', marker='s', ls='')
   
    ax.tick_params(labelsize=14, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    # ax.set_xticks([43, 43.5, 44, 44.5, 45, 45.5])

    ax.set_ylim(0, 1)
    ax.set_xlim(42.3, 45.7)

    if i > 2:
        ax.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=20)
    if i == 0 or i == 3:
        ax.set_ylabel(r'Completeness', fontsize=20)
    if i == 0:
        ax.legend(frameon=False)

fig.tight_layout()
fig.savefig('figures/Multi_comp.pdf', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 7), sharey=True)

puri_den_def = None
for i, ax in enumerate(axes.flatten()):
    [nb1, nb2] = nbs_list[i]

    z_min = (w_central[nb1] - nb_fwhm_Arr[nb1] * 0.5) / w_lya - 1
    z_max = (w_central[nb2] + nb_fwhm_Arr[nb2] * 0.5) / w_lya - 1

    # A text with the redshift interval
    ax.set_title(fr'$\bf z={z_min:0.1f}-{z_max:0.1f}$')

    this_dirname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'

    comp_list, _, _, puri_list, comp_den_list, _, _, puri_den_list, puricomp_bins = \
        load_puricomp1d(this_dirname)

    # Define the survey list in order
    survey_list = [f'AEGIS00{i}' for i in range(1, 4 + 1)] + ['J-NEP']

    # Bin centers
    bc = [puricomp_bins[i: i + 2].sum() * 0.5 for i in range(len(puricomp_bins) - 1)]

    # Plot the individual puris
    for j, puri in enumerate(puri_list):
        ax.plot(bc, puri, ls=':', alpha=0.6, marker='s',
                markersize=10, label=survey_list[j], c=f'C{j + 2}')

    # Total puri
    total_puri_num = (np.array(puri_list) *
                      np.array(puri_den_list)).sum(axis=0)
    total_puri_den = np.array(puri_den_list).sum(axis=0)
    total_puri = total_puri_num / total_puri_den
    total_puri_err = (total_puri_num / total_puri_den ** 2 +
                      total_puri_num ** 2 / total_puri_den ** 4 * total_puri_num) ** 0.5
    total_puri_err[~np.isfinite(total_puri_err)] = 0
    if puri_den_def is None:
        puri_den_def = total_puri_den
        puri_num_def = total_puri_num
    else:
        puri_den_def += total_puri_den
        puri_num_def += total_puri_num
    total_puri[~np.isfinite(total_puri)] = 0.
    ax.errorbar(bc, total_puri, ls='-', fmt='s', c='k',
                markersize=9, yerr=total_puri_err,
                capsize=4)
    np.save(f'npy/puri_1d_{nb1}-{nb2}', total_puri)
    np.save(f'npy/puri_1d_bc', bc)

    # Dummy point
    ax.errorbar([], [], markersize=9, c='k', label='Total', marker='s', ls='')

    ax.tick_params(labelsize=14, direction='in', which='both')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    # ax.set_xticks([43, 43.5, 44, 44.5, 45, 45.5])

    ax.set_ylim(0, 1)
    ax.set_xlim(42.3, 45.7)

    if i > 2:
        ax.set_xlabel(
            r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=20)
    if i == 0 or i == 3:
        ax.set_ylabel(r'Purity', fontsize=20)
    if i == 0:
        ax.legend(frameon=False)

fig.tight_layout()
fig.savefig('figures/Multi_puri.pdf', bbox_inches='tight', facecolor='w')
plt.show()


In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(10, 8))

fig.subplots_adjust(hspace=0.1)

total_puri_num = None

for i in range(6):
    [nb1, nb2] = nbs_list[i]

    this_dirname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
    comp_list, _, _, puri_list, comp_den_list, _, _, puri_den_list, puricomp_bins = \
        load_puricomp1d(this_dirname)

    if total_puri_num is None:
        total_puri_num = (np.array(puri_list) * np.array(puri_den_list))
        total_puri_den = np.array(puri_den_list)
        total_comp_num = (np.array(comp_list) * np.array(comp_den_list))
        total_comp_den = np.array(comp_den_list)
    else:
        total_puri_num += (np.array(puri_list) * np.array(puri_den_list))
        total_puri_den += np.array(puri_den_list)
        total_comp_num += (np.array(comp_list) * np.array(comp_den_list))
        total_comp_den += np.array(comp_den_list)

for i, puri in enumerate(puri_list):
    puri = total_puri_num[i] / total_puri_den[i]
    axs[0].plot(bc, puri, ls='--', alpha=0.6, marker='s', markersize=10,
                color=f'C{i + 2}', label=survey_list[i])

for i, comp in enumerate(comp_list):
    comp = total_comp_num[i] / total_comp_den[i]
    axs[1].plot(bc, comp, ls='--', alpha=0.6, marker='s', markersize=10,
                color=f'C{i + 2}')

combined_puri = puri_num_def / puri_den_def
combined_puri[~np.isfinite(combined_puri)] = 0.
combined_comp = comp_num_def / comp_den_def
combined_comp[~np.isfinite(combined_comp)] = 0.
total_comp_err = (comp_num_def / comp_den_def ** 2 +
                    comp_num_def ** 2 / comp_den_def ** 4 * comp_num_def) ** 0.5
total_comp_err[~np.isfinite(total_comp_err)] = 0
total_puri_err = (puri_num_def / puri_den_def ** 2 +
                    puri_num_def ** 2 / puri_den_def ** 4 * puri_num_def) ** 0.5
total_puri_err[~np.isfinite(total_puri_err)] = 0

axs[0].errorbar(bc, combined_puri, ls='-', fmt='s', color='black',
            markersize=10, label='Total', yerr=total_puri_err, capsize=4)
axs[1].errorbar(bc, combined_comp, ls='-', fmt='s', color='black',
            markersize=10, yerr=total_comp_err, capsize=4)

np.save(f'tmp/comb_puri_{qso_factor:0.1f}', combined_puri)
np.save(f'tmp/comb_puri_err_{qso_factor:0.1f}', total_puri_err)

# Font size
fs = 15

axs[0].legend(loc=0, fontsize=14)

axs[1].set_xlabel(r'$\log L$ (erg$\,$s$^{-1}$)', fontsize=20)
axs[0].set_ylabel('Purity', fontsize=20)
axs[1].set_ylabel('Completeness', fontsize=20)

for ax in axs:
    ax.tick_params(labelsize=fs, direction='in', length=6)
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    ax.set_ylim(0, 1)
    ax.set_xlim(42.3, 45.7)

# fig.tight_layout()

fig.savefig('figures/Combined_puricomp.pdf', bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
this_L_min = 43.5
what_bins = (LF_dict['LF_bins'] > this_L_min)
what_bins_comp = (np.array(bc) > this_L_min)
N_LAEs = (LF_dict['LF_total'] * volwid)[what_bins].sum() / 1.14
this_total_comp = comp_num_def[what_bins_comp].sum() / comp_den_def[what_bins_comp].sum()
print(f'N_LAEs = {N_LAEs:0.1f}')
print(f'median comp = {this_total_comp:0.2f}')

In [ ]:
def load_qsofrac_LF(qso_factor):
    # My LF

    # nbs_list = [[20, 24]]
    nbs_list = [[1, 4], [4, 8], [8, 12], [12, 16], [16, 20], [20, 24]]

    survey_list = [f'minijpasAEGIS00{i}' for i in range(1, 4 + 1)] + ['jnep']
    total_volume = 0
    for [this_nb_min, this_nb_max] in nbs_list:
        total_volume += effective_volume(this_nb_min, this_nb_max, 'both')
    L_binning = np.load('npy/L_nb_err_binning.npy')
    b = np.log10(L_binning)
    LF_bins = np.array([(b[i] + b[i + 1]) / 2 for i in range(len(b) - 1)])
    bin_width = np.array([b[i + 1] - b[i] for i in range(len(b) - 1)])

    hist_mat = None
    LF_raw = None
    for i, [nb1, nb2] in enumerate(nbs_list):
        this_volume = effective_volume(nb1, nb2)
        this_hist = None
        for survey_name in survey_list:
            pathname = f'Luminosity_functions/LF_r17-24_nb{nb1}-{nb2}_ew30_ewoth100_nb_{qso_factor:0.1f}'
            filename_hist = f'{pathname}/hist_i_mat_{survey_name}.npy'
            hist_i_mat = np.load(filename_hist)

            if this_hist is None:
                this_hist = hist_i_mat
            else:
                this_hist += hist_i_mat
        # When all 5 fields are added up, divide by total volume
        this_hist = this_hist / total_volume / bin_width

        if hist_mat is None:
            hist_mat = this_hist
        else:
            # hist_mat = np.vstack([hist_mat, this_hist])
            hist_mat = hist_mat + this_hist

        filename_dict = f'{pathname}/LFs.pkl'
        with open(filename_dict, 'rb') as file:
            if LF_raw is None:
                LF_raw = pickle.load(file)['LF_total_raw'] * this_volume
            else:
                LF_raw += pickle.load(file)['LF_total_raw'] * this_volume
        
    L_LF_err_percentiles = np.percentile(hist_mat, [16, 50, 84], axis=0)
    LF_err_plus = L_LF_err_percentiles[2] - L_LF_err_percentiles[1]
    LF_err_minus = L_LF_err_percentiles[1] - L_LF_err_percentiles[0]
    hist_median = L_LF_err_percentiles[1]

    volwid = total_volume * bin_width
    yerr_plus = (hist_median + volwid * (LF_err_plus) ** 2) ** 0.5 * volwid ** -0.5
    yerr_minus = (hist_median + volwid * (LF_err_minus) ** 2) ** 0.5 * volwid ** -0.5

    LF_dict = {
        'LF_bins': LF_bins,
        'LF_total': hist_median,
        'LF_total_uncorr': LF_raw / total_volume,
        'LF_total_err': [yerr_minus, yerr_plus]
    }
    
    return LF_dict

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(12, 4.2), sharey='row', sharex=True,
                        gridspec_kw=dict(height_ratios=[1.7, 1]))

fig.subplots_adjust(hspace=0.12)
fig.subplots_adjust(wspace=0.05)

axs0 = axs[0]
axs1 = axs[1]

Lx = np.linspace(10 ** 42, 10 ** 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = schechter(Lx, phistar1, 10 ** Lstar1, alpha1) * Lx * np.log(10)

for iii, qso_frac in enumerate([0.5, 0.8, 1.0, 1.2]):
    this_LF_dict = load_qsofrac_LF(qso_frac)

    axs0[iii].plot(
        np.log10(Lx), Phi_center * qso_frac, ls='-', alpha=1,
        label='Input LF', zorder=51,
        color='g'
    )
    axs0[iii].errorbar(this_LF_dict['LF_bins'], this_LF_dict['LF_total'],
            yerr=this_LF_dict['LF_total_err'][:2],
            # xerr=bin_width*0.5,
            linestyle='', fmt='s',
            ecolor='k', markeredgecolor='k', markerfacecolor='red',
            markeredgewidth=1, markersize=6,
            capsize=4, zorder=99)

    axs0[iii].set_ylim(1e-8, 1e-4)
    axs0[iii].set_xlim(42.3, 45.7)
    axs0[iii].set_yscale('log')
    axs0[iii].tick_params(direction='in', which='both', labelsize=11)
    axs0[iii].yaxis.set_ticks_position('both')
    axs0[iii].xaxis.set_ticks_position('both')
    if iii == 0:
        axs0[iii].set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)', fontsize=12)

    axs0[iii].set_title(fr'$\rho={qso_frac:0.1f}$', fontsize=13)
    axs0[iii].legend(fontsize=10)

    #### PURICOMP ####

    this_puri = np.load(f'tmp/comb_puri_{qso_frac:0.1f}.npy')
    this_puri_err = np.load(f'tmp/comb_puri_err_{qso_frac:0.1f}.npy')

    axs1[iii].errorbar(bc, this_puri, ls='-', c='k', fmt='s',
                       yerr=this_puri_err, capsize=4)

    axs1[iii].set_ylim(0, 1)
    axs1[iii].tick_params(direction='in', which='both', labelsize=11)
    axs1[iii].set_yticks(np.arange(0, 1.25, 0.25))
    axs1[iii].set_yticklabels(['0.0', '', '0.5', '', '1.0'])
    axs1[iii].yaxis.set_ticks_position('both')
    axs1[iii].xaxis.set_ticks_position('both')

    axs1[iii].set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=12)
    if iii == 0:
        axs1[iii].set_ylabel('Purity', fontsize=12)
    ##################

fig.savefig('figures/rho_LF_puri.pdf', bbox_inches='tight', pad_inches=0,
            facecolor='w')
plt.show()

In [ ]:
fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(5, 5.5), sharex=True,
                        gridspec_kw=dict(height_ratios=[2, 1]))

fig.subplots_adjust(hspace=0.12)

Lx = np.linspace(10 ** 42, 10 ** 46, 10000)
phistar1 = 3.33e-6
Lstar1 = 44.65
alpha1 = -1.35

phistar2 = -3.45
Lstar2 = 42.93
alpha2 = -1.93

Phi_center = schechter(Lx, phistar1, 10 ** Lstar1, alpha1) * Lx * np.log(10)

for ii, qso_frac in enumerate([0.5, 0.8, 1, 1.2]):
    iii = ii + 7
    this_LF_dict = load_qsofrac_LF(qso_frac)

    ax0.plot(
        np.log10(Lx), Phi_center * qso_frac, ls='-', alpha=1,
        label=rf'$\rho = {qso_frac}$', zorder=51,
        color=f'C{iii}'
    )
    ax0.errorbar(this_LF_dict['LF_bins'], this_LF_dict['LF_total'],
            yerr=(this_LF_dict['LF_total_err'][:2] if iii == 3 else None),
            # xerr=bin_width*0.5,
            linestyle='', fmt='s',
            ecolor=f'C{iii}', markeredgecolor=f'C{iii}', markerfacecolor='none',
            markeredgewidth=1.5, markersize=6,
            capsize=4, zorder=99-iii)

    #### PURICOMP ####

    this_puri = np.load(f'tmp/comb_puri_{qso_frac:0.1f}.npy')
    this_puri_err = np.load(f'tmp/comb_puri_err_{qso_frac:0.1f}.npy')

    ax1.errorbar(bc, this_puri, ls='-', c=f'C{iii}', fmt='',
                       yerr=this_puri_err, capsize=0)

    ax1.set_ylim(0, 1)
    ax1.tick_params(direction='in', which='both', labelsize=11)
    ax1.set_yticks(np.arange(0, 1.25, 0.25))
    ax1.set_yticklabels(['0.0', '', '0.5', '', '1.0'])
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')

    ##################

    ax0.set_ylim(1e-8, 5e-4)
    ax0.set_xlim(42.3, 45.7)
    ax0.set_yscale('log')
    ax0.tick_params(direction='in', which='both', labelsize=11)
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.set_ylabel(r'$\Phi$ (Mpc$^{-3}\,\Delta\logL^{-1}$)', fontsize=12)
    ax0.legend(fontsize=10, loc=0)

    ax1.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$\,$s$^{-1}$)', fontsize=12)
    ax1.set_ylabel('Purity', fontsize=12)

fig.savefig('figures/rho_LF_puri_one.png', bbox_inches='tight',
            facecolor='w')
plt.show()